## Build CNN Model 

In [1]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
import tensorflow as tf


In [3]:
print(tf.__version__)

1.15.5


In [4]:
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
mnist.train.num_examples

55000

In [7]:
mnist.test.num_examples

10000

## Define Weights

In [8]:
def initialize_weights (filter_shape):
    init_random_dist = tf.truncated_normal(filter_shape, stddev=0.1)
    
    return ( tf.Variable(init_random_dist) )

## Define Bias

In [9]:
def initialize_bias (bias_shape):
    
    initial_bias_vals = tf.constant (0.1, shape = bias_shape)
    
    return (tf.Variable (initial_bias_vals))

## Define Convolution Layer

In [10]:
def create_convolution_layer_and_compute_dot_product (inputs, filter_shape):
    
    filter_initialized_with_weights = initialize_weights (filter_shape)
    
    conv_layer_outputs = tf.nn.conv2d(inputs, filter_initialized_with_weights, strides = [1, 1, 1, 1], padding='SAME')
    
    return (conv_layer_outputs)

## Define ReLu Layer 

In [11]:
def create_relu_layer_and_compute_dotproduct_plus_b(inputs, filter_shape):
    
    b = initialize_bias ([ filter_shape[3] ])
    
    relu_layer_outputs = tf.nn.relu(inputs + b)
    
    return ( relu_layer_outputs )

## Define Max pool layer of 2 by 2 

In [12]:
def create_maxpool2by2_and_reduce_spatial_size (inputs): 
    
    pooling_layer_outputs = tf.nn.max_pool(inputs, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    return ( pooling_layer_outputs )

## Define Fully connected Layer 

In [13]:
def create_fully_connected_layer_and_compute_dotproduct_plus_bias (inputs, output_size):
    
    input_size = int(inputs.get_shape()[1])
    
    
    W = initialize_weights ([input_size, output_size])
    
    b = initialize_bias ([output_size])
    
    fc_xW_plus_bias_outputs = tf.matmul(inputs, W) + b
    
    return (fc_xW_plus_bias_outputs)

## Build CNN Model 

In [14]:
x = tf.placeholder(tf.float32, shape=[None, 784])

In [15]:
y_true = tf.placeholder(tf.float32, [None, 10])

In [16]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [17]:
conv_layer_1_outputs \
    = create_convolution_layer_and_compute_dot_product (x_image, filter_shape=[5, 5, 1, 32])



conv_relu_layer_1_outputs \
    = create_relu_layer_and_compute_dotproduct_plus_b (conv_layer_1_outputs, filter_shape=[5, 5, 1, 32])

In [18]:
pooling_layer_1_outputs = create_maxpool2by2_and_reduce_spatial_size (conv_relu_layer_1_outputs)

In [21]:
pooling_layer_1_outputs_flat = tf.reshape(pooling_layer_1_outputs, [-1, 14 * 14 * 32])

In [22]:
fc_layer_1_outputs \
    = create_fully_connected_layer_and_compute_dotproduct_plus_bias (pooling_layer_1_outputs_flat, output_size = 1024)


fc_relu_layer_1_outputs = tf.nn.relu ( fc_layer_1_outputs )

In [23]:
hold_prob = tf.placeholder(tf.float32)


fc_dropout_outputs = tf.nn.dropout(fc_relu_layer_1_outputs, keep_prob=hold_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [24]:
y_pred = create_fully_connected_layer_and_compute_dotproduct_plus_bias(fc_dropout_outputs, output_size=10)

In [25]:
softmax_cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits = y_pred)

cross_entropy_mean = tf.reduce_mean(softmax_cross_entropy_loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [26]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)

In [27]:
cnn_trainer = optimizer.minimize(cross_entropy_mean)

In [28]:
vars_initializer = tf.global_variables_initializer()

## Train and Test CNN Model 

In [29]:
steps = 5000

In [30]:
with tf.Session() as sess: 
    
    sess.run(vars_initializer)
    
    for i in range(steps):
        
        batch_x, batch_y = mnist.train.next_batch(50)
        
        sess.run(cnn_trainer, feed_dict={x:batch_x, y_true: batch_y, hold_prob: 0.5})
        
        if i % 100 == 0:
            print('ON STEP: {}'.format(i))
            print('ACCURACY: ')
            
            
            matches = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
            
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            
            
            test_accuracy = sess.run ( acc, feed_dict = {x: mnist.test.images, \
                                                        y_true: mnist.test.labels, \
                                                        hold_prob: 1.0})
            
            print(test_accuracy)
            print('\n')


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   OMP_NUM_THREADS=8

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_HAND_THREAD=false
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_IN

ON STEP: 0
ACCURACY: 
0.1089


ON STEP: 100
ACCURACY: 
0.8882


ON STEP: 200
ACCURACY: 
0.9254


ON STEP: 300
ACCURACY: 
0.9416


ON STEP: 400
ACCURACY: 
0.9454


ON STEP: 500
ACCURACY: 
0.9616


ON STEP: 600
ACCURACY: 
0.9648


ON STEP: 700
ACCURACY: 
0.9692


ON STEP: 800
ACCURACY: 
0.9701


ON STEP: 900
ACCURACY: 
0.9715


ON STEP: 1000
ACCURACY: 
0.9778


ON STEP: 1100
ACCURACY: 
0.9785


ON STEP: 1200
ACCURACY: 
0.9794


ON STEP: 1300
ACCURACY: 
0.9775


ON STEP: 1400
ACCURACY: 
0.9778


ON STEP: 1500
ACCURACY: 
0.9827


ON STEP: 1600
ACCURACY: 
0.9805


ON STEP: 1700
ACCURACY: 
0.9818


ON STEP: 1800
ACCURACY: 
0.9818


ON STEP: 1900
ACCURACY: 
0.9812


ON STEP: 2000
ACCURACY: 
0.9821


ON STEP: 2100
ACCURACY: 
0.983


ON STEP: 2200
ACCURACY: 
0.9844


ON STEP: 2300
ACCURACY: 
0.9852


ON STEP: 2400
ACCURACY: 
0.9851


ON STEP: 2500
ACCURACY: 
0.9868


ON STEP: 2600
ACCURACY: 
0.9854


ON STEP: 2700
ACCURACY: 
0.9851


ON STEP: 2800
ACCURACY: 
0.9856


ON STEP: 2900
ACCURACY: 
0.